In [1]:
import pandas as pd
import pymongo
import config
import database as db
import matplotlib.pyplot as plt
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

In [14]:
def get_market_year(row):
    year = row['Year']
    week = row['Week']
    culture = row['Culture']
    if culture in ['Blé tendre', 'Blé dur']:
        if week >= 36:
            market_year = f"{year}/{year + 1}"
        else:
            market_year = f"{year - 1}/{year}"
    else:
        if 9 <= week <= 49:
            market_year = f"{year - 1}/{year}"
        else:
            market_year = f"{year - 1}/{year}"
    return market_year

def get_market_year_ble(row):
    year = row['Year']
    week = row['Week']
    if week >= 36:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [15]:
dfMeteo = pd.read_csv('C:/Users/alexl/Dropbox/Alex/aleexv6Corporation/Trading/Meteo/nouvelle_aquitaine_total.csv', parse_dates=['DATE'])
cursorDev = db.get_database_dev_cond_region().find({})
dfCondition = pd.DataFrame(list(cursorDev)).sort_values(by='Date', ascending=True)

In [16]:
dfConditionNA = dfCondition[(dfCondition['Région'] == 'Nouvelle-Aquitaine') & ((dfCondition['Date'].dt.year >= 2015) & (dfCondition['Date'].dt.year < 2024))]
dfConditionNA['Week'] = dfConditionNA['Semaine'].str.extract(r'S(\d+)')[0].astype(int)
dfConditionNA['Year'] = dfConditionNA['Semaine'].str.extract(r'(\d+)-S\d+')[0].astype(int)
dfConditionNA['MarketYear'] = dfConditionNA.apply(get_market_year, axis=1)
dfConditionNA = dfConditionNA[['Date','Culture', 'MarketYear','Très mauvaises','Mauvaises','Assez bonnes', 'Bonnes', 'Très bonnes']]

dfMeteo['TX'] = dfMeteo['TX'].replace(',', '.', regex=True).astype(float)
dfMeteo['TN'] = dfMeteo['TN'].replace(',', '.', regex=True).astype(float)
dfMeteo['RR'] = dfMeteo['RR'].replace(',', '.', regex=True).astype(float)
dfMeteo['DG'] = dfMeteo['DG'].replace(',', '.', regex=True).astype(float)
dfMeteo['Year'] = dfMeteo['DATE'].dt.year
dfMeteo['Week'] = dfMeteo['DATE'].dt.isocalendar().week
dfMeteo['MarketYear'] = dfMeteo.apply(get_market_year_ble, axis=1)
dfMeteo = dfMeteo[['POSTE', 'DEPARTEMENT', 'DATE', 'Week', 'MarketYear', 'TX', 'TN', 'RR', 'DG']]

In [26]:
dfMeteoMoyenneNA = dfMeteo[(dfMeteo['MarketYear'] == '2021/2022') & (dfMeteo['Week'] < 52)].groupby('DATE').mean()
dfMeteoMoyenneNA

,POSTE,DEPARTEMENT,Week,TX,TN,RR,DG
DATE,,,,,,,
2021-09-06,4.662023e+07,46.418605,36.0,32.385427,14.540704,0.014151,0.0
2021-09-07,4.662023e+07,46.418605,36.0,32.298995,16.984925,0.042925,0.0
2021-09-08,4.662023e+07,46.418605,36.0,27.116080,18.434171,8.461792,0.0
2021-09-09,4.662023e+07,46.418605,36.0,23.891960,16.812563,6.893396,0.0
2021-09-10,4.662023e+07,46.418605,36.0,23.752764,16.168844,3.248585,0.0
...,...,...,...,...,...,...,...
2022-08-31,4.670218e+07,46.500000,35.0,29.258173,15.953846,2.185377,0.0
2022-09-01,4.670218e+07,46.500000,35.0,28.266346,16.330769,3.166509,0.0
2022-09-02,4.670218e+07,46.500000,35.0,25.263158,15.982297,4.441981,0.0
